In [1]:
# NOTE RATING IS 0-5 INTEGER ONLY

In [3]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from itertools import islice
import csv

In [4]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [7]:
def readCSV(path):
    with open(path, mode='r') as file:
        reader = csv.reader(file)
        # Skip the header row
        next(reader, None)
        # Loop through the rows
        for row in reader:
            # Extract the first 4 columns
            u,i,d,r = row[:4]
            yield u, i, d, float(r)


### This is the Alternative Model
Feel free to play with the data! To avoid merge conflicts though either duplicate the file or use a different branch.
If there are any issues or fixes to stuff that I've done lmk.

The baseline is just here to chill around. 


In [8]:
train_dataset = []
ratings_dict = defaultdict(int)
itemsPerUser = defaultdict(set)
usersPerItem = defaultdict(set)
global_fallback_rating = 0
for l in readCSV("data/interactions_train.csv"):
    train_dataset.append(l)
    user, recipe, data, rating = l
    itemsPerUser[user].add(recipe)
    usersPerItem[recipe].add(user)
    global_fallback_rating += rating

In [9]:
global_fallback_rating = global_fallback_rating / len(train_dataset)
global_fallback_rating
def baseline_predictor(user, item, date):
    return global_fallback_rating

In [10]:
validation_dataset = []
validation_reals = []
validation_preds = []
for l in readCSV("data/interactions_validation.csv"):
    validation_dataset.append(l)
    user, recipe, date, rating = l
    validation_reals.append(rating)
    validation_preds.append(baseline_predictor(user, recipe, date))
print(f'baseline MSE: {mean_squared_error(validation_reals, validation_preds)}')

baseline MSE: 1.8138061805801156


### Alternative Model

This is a simple SVDpp using purely relationships between rated recipes and user ids, similar to the data format of assignment 1. 
The n_factors yield the best result at 1 from Cross-Validation. Any substantially higher number (e.g 50) will take too much time to run given a specific run time with the dataset. 

In [37]:
#This is a model using svdpp to compare our original model to
import gzip
from collections import defaultdict
import random
import csv
import numpy as np
import math
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVDpp
from surprise.model_selection import train_test_split
from surprise import accuracy

#using pandas to make display a bit easier to debug
df = pd.read_csv("data/interactions_train.csv", names=['user_id', 'recipe', 'date', 'rating', "u", "i"], skiprows = 1 )
print(df[0:10])
df['rating'] = df['rating'].astype(int)  
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'recipe','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.001)
model = SVDpp(n_factors=1, lr_all=0.01, reg_all = 0.1)
model.fit(trainset)

#validation
pairs = pd.read_csv('data/interactions_validation.csv', names=['user_id', 'recipe', 'date', 'rating', "u", "i"], skiprows=1)
predictions = []
for _, row in pairs.iterrows():
    user = row['user_id']
    rec = row['recipe']
    pred = model.predict(user, rec).est
    predictions.append(pred - 0.25)

#test
pairsT = pd.read_csv('data/interactions_test.csv', names=['user_id', 'recipe', 'date', 'rating', "u", "i"], skiprows=1)
predictionsT = []
for _, row in pairsT.iterrows():
    user = row['user_id']
    rec = row['recipe']
    pred = model.predict(user, rec).est
    predictionsT.append(pred- 0.25)
    
    
pairs['prediction'] = predictions
pairsT['prediction'] = predictionsT

predictions_ttest = model.test(testset)
mse = accuracy.mse(predictions_ttest)
print(f'MSE on train / test split: {mse}')

validation_actual = pairs['rating'].astype(float)  
validation_predicted = pairs['prediction']        
validation_mse = np.mean((validation_actual - validation_predicted) ** 2)
print(f'MSE on validation set: {validation_mse}')

test_actual = pairsT['rating'].astype(float)
test_predicted = pairsT['prediction']
test_mse = np.mean((test_actual - test_predicted) ** 2)
print(f'MSE on test set: {test_mse}')

   user_id  recipe        date  rating      u       i
0     2046    4684  2000-02-25     5.0  22095   44367
1     2046     517  2000-02-25     5.0  22095   87844
2     1773    7435  2000-03-13     5.0  24732  138181
3     1773     278  2000-03-13     4.0  24732   93054
4     2046    3431  2000-04-07     5.0  22095  101723
5     2046   13307  2000-05-21     5.0  22095  134551
6     2312     780  2000-09-12     5.0   1674  127175
7     2312   51964  2000-09-26     5.0   1674  151793
8     2312    1232  2000-10-17     4.0   1674   15498
9     2312    4397  2000-10-17     5.0   1674   14380
MSE: 0.8488
MSE on train / test split: 0.8488063963281915
MSE on validation set: 1.549737618724458
MSE on test set: 1.651565567137675


In [53]:
#this is to play around with shifters

testpr = []
for i in predictions:
    testpr.append(i)
ttestpr = []
for i in predictionsT:
    ttestpr.append(i)
    
nvalidation_actual = pairs['rating'].astype(float)
nvalidation_predicted = testpr
nvalidation_mse = np.mean((nvalidation_actual - nvalidation_predicted) ** 2)
print(f'MSE on validation set: {nvalidation_mse}')


ntest_actual = pairsT['rating'].astype(float)
ntest_predicted = ttestpr
ntest_mse = np.mean((ntest_actual - ntest_predicted) ** 2)
print(f'MSE on test set: {ntest_mse}')

MSE on validation set: 1.5500290567556492
MSE on test set: 1.651565567137675


In [32]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection import RandomizedSearchCV
'''param_grid = {
    'n_factors': [7, 15],  
    'lr_all': [0.005, 0.01],
    'reg_all': [0.1, 0.2]
}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=2)
gs.fit(data)

results = gs.cv_results
'''
#randomized CV to cut processing time
param_grid = {
    'n_factors': [7, 15],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.2, 0.3]
}
rs = RandomizedSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3, n_iter=10)  # Only test 10 random combinations
rs.fit(data)
print("Best NF", rs.best_params['rmse'])
results = rs.cv_results

for params, mean_rmse, std_rmse in zip(results['params'], results['mean_test_rmse'], results['std_test_rmse']):
    print(f"Params: {params}, Mean RMSE: {mean_rmse:.4f}, Std Dev RMSE: {std_rmse:.4f}")
    

KeyboardInterrupt: 